In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity = np.zeros(shape=(94,60000)) #100个agent的资源量 定义成一个矩阵 100*20000
dual_variable = np.zeros(shape=(94,60000))#对偶变量
x_average_resource_quantity=np.zeros(shape=(94,60000)) #动态一致平均算法追踪平均值的变量
average_resource=np.zeros(shape=(1,60000))#存放每一次迭代后所有用户资源量的平均值
average_cost=np.zeros(shape=(1,60000))#存放每一次迭代后所有用户成本的平均值
iteration_update = 0.1
a=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b=np.loadtxt("100_normal_distribution_(2,0.5).txt")#每个agent的成本函数中的超参数
W=np.zeros(shape=(94,94)) #双随机的网络拓扑矩阵W（对角线处非零）
iteration=[]#绘图横坐标

In [3]:
for i in range(0,94):
    for j in range(0,94):
        W[i][j]=1/94
        
#对偶变量的初始化 初始化一致
for i in range (0,94):
    dual_variable[i][0]=0
    
#资源量的初始化 资源量的初始化需要和对偶变量的初始值满足某种关系
for i in range (0,94):
    agent_resource_quantity[i][0]=0
    
#追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,94):
    x_average_resource_quantity[i][0]=agent_resource_quantity[i][0]

iteration.append(0)

#诚实用户平均成本初始化
sum=0
for i in range(0,94):
    sum=sum+a[i]*(agent_resource_quantity[i][0]-b[i])*(agent_resource_quantity[i][0]-b[i])
average_cost[0][0]=sum/94

In [4]:
#DRA算法 
def DRA(a,b,agent_resource_quantity,average_resource,iteration_update,dual_variable,W,iteration,x_average_resource_quantity):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DRA算法+无拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,94):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量平均值avg为:",average_resource[0][0])
    print("*************************")
    print("初始平均成本为:",average_cost[0][0])
    print("*************************")
    for i in range(0,94):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable[i][0])
    print("*************************")
    for i in range(0,94):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration.append(k) 
        
        #对偶变量的迭代
        for i in range(0,94):
            dual_variable[i][k]=dual_variable[i][k-1]+iteration_update*((x_average_resource_quantity[i][k-1])-(50))
        
        #用户资源申请量的迭代
        for i in range (0,94):
            agent_resource_quantity[i][k]=agent_resource_quantity[i][k-1]-iteration_update*(((2*a[i]*agent_resource_quantity[i][k-1]-2*a[i]*b[i])/94)+(dual_variable[i][k-1]/94))
            #原变量的投影操作
            if agent_resource_quantity[i][k] < 0:
                agent_resource_quantity[i][k]=0
            elif agent_resource_quantity[i][k] > 100: 
                agent_resource_quantity[i][k]=100
                
        #平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (0,94):
            sum_x=0
            for j in range (0,94):
                if W[i][j]!=0:
                    sum_x=sum_x+W[i][j]*x_average_resource_quantity[j][k-1]
            x_average_resource_quantity[i][k]=sum_x+(agent_resource_quantity[i][k]-agent_resource_quantity[i][k-1])                    
        
        #计算所有诚实用户的平均资源量
        sum1=0
        for i in range (0,94):
            sum1=sum1+agent_resource_quantity[i][k]
        average_resource[0][k]=(sum1/94)
        
        #计算所有诚实用户的平均成本
        sum2=0
        for i in range(0,94):
            sum2=sum2+a[i]*(agent_resource_quantity[i][k]-b[i])*(agent_resource_quantity[i][k]-b[i])
        average_cost[0][k]=sum2/94
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,94):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
            print("资源申请量平均值avg为:",average_resource[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost[0][k])
            print("****************************************************************************")
            for i in range(0,94):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable[i][k]) 
            print("****************************************************************************")
            for i in range(0,94):
                print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity[i][k])
            print("*************************")
            print("迭代步长为:",iteration_update)
            print("****************************************************************************")
        
        #iteration_update=1/(pow(k+1,0.5)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
DRA(a,b,agent_resource_quantity,average_resource,iteration_update,dual_variable,W,iteration,x_average_resource_quantity)

****************************DRA算法+无拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请量为

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 55.70768492258146
agent 2 在 3999 次迭代中的资源申请量为: 55.67212594011918
agent 3 在 3999 次迭代中的资源申请量为: 60.92350150299501
agent 4 在 3999 次迭代中的资源申请量为: 39.61433799098883
agent 5 在 3999 次迭代中的资源申请量为: 56.63824202994966
agent 6 在 3999 次迭代中的资源申请量为: 54.562389722186495
agent 7 在 3999 次迭代中的资源申请量为: 66.39070625963797
agent 8 在 3999 次迭代中的资源申请量为: 46.69458717065563
agent 9 在 3999 次迭代中的资源申请量为: 65.37646757803547
agent 10 在 3999 次迭代中的资源申请量为: 54.53367875583724
agent 11 在 3999 次迭代中的资源申请量为: 66.66153117217861
agent 12 在 3999 次迭代中的资源申请量为: 38.12406783808906
agent 13 在 3999 次迭代中的资源申请量为: 51.093586039271855
agent 14 在 3999 次迭代中的资源申请量为: 41.48784745767473
agent 15 在 3999 次迭代中的资源申请量为: 42.82198539471635
agent 16 在 3999 次迭代中的资源申请量为: 49.04844317495326
agent 17 在 3999 次迭代中的资源申请量为: 49.553850775536255
agent 18 在 3999 次迭代中的资源申请量为: 46.198905443519635
agent 19 在 3999 次迭代中的资源申请量为: 45.6899158613204
agent 20 在 3999 次迭代中的资源申请量为: 50.1129492815981
agent 21 在 3999 次迭代中的资源申请量为: 69.16065455153357
agent 

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 55.58754606716252
agent 2 在 5999 次迭代中的资源申请量为: 55.55194668238251
agent 3 在 5999 次迭代中的资源申请量为: 60.80409466497853
agent 4 在 5999 次迭代中的资源申请量为: 39.4933744251467
agent 5 在 5999 次迭代中的资源申请量为: 56.518242363757444
agent 6 在 5999 次迭代中的资源申请量为: 54.4421323843543
agent 7 在 5999 次迭代中的资源申请量为: 66.2725819778456
agent 8 在 5999 次迭代中的资源申请量为: 46.57362435220466
agent 9 在 5999 次迭代中的资源申请量为: 65.25808881009404
agent 10 在 5999 次迭代中的资源申请量为: 54.41335813291822
agent 11 在 5999 次迭代中的资源申请量为: 66.5435400601897
agent 12 在 5999 次迭代中的资源申请量为: 38.00308786764553
agent 13 在 5999 次迭代中的资源申请量为: 50.97294056094325
agent 14 在 5999 次迭代中的资源申请量为: 41.36676276330795
agent 15 在 5999 次迭代中的资源申请量为: 42.700895748560995
agent 16 在 5999 次迭代中的资源申请量为: 48.927663016022876
agent 17 在 5999 次迭代中的资源申请量为: 49.43310445262399
agent 18 在 5999 次迭代中的资源申请量为: 46.07795878104375
agent 19 在 5999 次迭代中的资源申请量为: 45.5689177500228
agent 20 在 5999 次迭代中的资源申请量为: 49.99225220222156
agent 21 在 5999 次迭代中的资源申请量为: 69.04372910591765
agent 22 在

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 55.585488748031516
agent 2 在 7999 次迭代中的资源申请量为: 55.549901223498956
agent 3 在 7999 次迭代中的资源申请量为: 60.80189909329781
agent 4 在 7999 次迭代中的资源申请量为: 39.492073456784674
agent 5 在 7999 次迭代中的资源申请量为: 56.516150345551246
agent 6 在 7999 次迭代中的资源申请量为: 54.44010685731571
agent 7 在 7999 次迭代中的资源申请量为: 66.27027336540446
agent 8 在 7999 次迭代中的资源申请量为: 46.571913402236234
agent 9 在 7999 次迭代中的资源申请量为: 65.25579444179829
agent 10 在 7999 次迭代中的资源申请量为: 54.41135323172707
agent 11 在 7999 次迭代中的资源申请量为: 66.54122321540922
agent 12 在 7999 次迭代中的资源申请量为: 38.00177351287818
agent 13 在 7999 次迭代中的资源申请量为: 50.9710515186327
agent 14 在 7999 次迭代中的资源申请量为: 41.36530343860968
agent 15 在 7999 次迭代中的资源申请量为: 42.69937049223205
agent 16 在 7999 次迭代中的资源申请量为: 48.92583619620078
agent 17 在 7999 次迭代中的资源申请量为: 49.431260734481725
agent 18 在 7999 次迭代中的资源申请量为: 46.076234225079
agent 19 在 7999 次迭代中的资源申请量为: 45.567237236690104
agent 20 在 7999 次迭代中的资源申请量为: 49.99038513171042
agent 21 在 7999 次迭代中的资源申请量为: 69.04136892054052
agen

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 55.58577676713129
agent 2 在 9999 次迭代中的资源申请量为: 55.55018938916586
agent 3 在 9999 次迭代中的资源申请量为: 60.802185217626445
agent 4 在 9999 次迭代中的资源申请量为: 39.492366252540066
agent 5 在 9999 次迭代中的资源申请量为: 56.51643792165818
agent 6 在 9999 次迭代中的资源申请量为: 54.44039526433663
agent 7 在 9999 次迭代中的资源申请量为: 66.27055764705777
agent 8 在 9999 次迭代中的资源申请量为: 46.572204747274235
agent 9 在 9999 次迭代中的资源申请量为: 65.25607897519464
agent 10 在 9999 次迭代中的资源申请量为: 54.411641881067894
agent 11 在 9999 次迭代中的资源申请量为: 66.54150735069837
agent 12 在 9999 次迭代中的资源申请量为: 38.00206630321405
agent 13 在 9999 次迭代中的资源申请量为: 50.97134139873453
agent 14 在 9999 次迭代中的资源申请量为: 41.36559598909898
agent 15 在 9999 次迭代中的资源申请量为: 42.69966282389578
agent 16 在 9999 次迭代中的资源申请量为: 48.92612664642451
agent 17 在 9999 次迭代中的资源申请量为: 49.43155103610127
agent 18 在 9999 次迭代中的资源申请量为: 46.07652547623012
agent 19 在 9999 次迭代中的资源申请量为: 45.567528781053774
agent 20 在 9999 次迭代中的资源申请量为: 49.99067522034502
agent 21 在 9999 次迭代中的资源申请量为: 69.0416522394971
agen

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 55.585782035080285
agent 2 在 51999 次迭代中的资源申请量为: 55.55019462821262
agent 3 在 51999 次迭代中的资源申请量为: 60.802190824507036
agent 4 在 51999 次迭代中的资源申请量为: 39.49236966786117
agent 5 在 51999 次迭代中的资源申请量为: 56.51644327427469
agent 6 在 51999 次迭代中的资源申请量为: 54.440400454691755
agent 7 在 51999 次迭代中的资源申请量为: 66.27056354477378
agent 8 在 51999 次迭代中的资源申请量为: 46.57220916854672
agent 9 在 51999 次迭代中的资源申请量为: 65.25608483377385
agent 10 在 51999 次迭代中的资源申请量为: 54.41164702113957
agent 11 在 51999 次迭代中的资源申请量为: 66.54151327090639
agent 12 在 51999 次迭代中的资源申请量为: 38.00206975142279
agent 13 在 51999 次迭代中的资源申请量为: 50.971346255829985
agent 14 在 51999 次迭代中的资源申请量为: 41.365599793304895
agent 15 在 51999 次迭代中的资源申请量为: 42.69966678990019
agent 16 在 51999 次迭代中的资源申请量为: 48.92613135135689
agent 17 在 51999 次迭代中的资源申请量为: 49.43155578236967
agent 18 在 51999 次迭代中的资源申请量为: 46.076529930827164
agent 19 在 51999 次迭代中的资源申请量为: 45.56753312775292
agent 20 在 51999 次迭代中的资源申请量为: 49.99068002371856
agent 21 在 51999 次迭代中的资源申请量为

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 55.58578203512547
agent 2 在 53999 次迭代中的资源申请量为: 55.55019462825747
agent 3 在 53999 次迭代中的资源申请量为: 60.80219082455656
agent 4 在 53999 次迭代中的资源申请量为: 39.492369667891765
agent 5 在 53999 次迭代中的资源申请量为: 56.51644327432089
agent 6 在 53999 次迭代中的资源申请量为: 54.440400454736064
agent 7 在 53999 次迭代中的资源申请量为: 66.27056354482775
agent 8 在 53999 次迭代中的资源申请量为: 46.57220916858386
agent 9 在 53999 次迭代中的资源申请量为: 65.25608483382717
agent 10 在 53999 次迭代中的资源申请量为: 54.41164702118333
agent 11 在 53999 次迭代中的资源申请量为: 66.54151327096076
agent 12 在 53999 次迭代中的资源申请量为: 38.00206975145356
agent 13 在 53999 次迭代中的资源申请量为: 50.97134625587087
agent 14 在 53999 次迭代中的资源申请量为: 41.36559979333774
agent 15 在 53999 次迭代中的资源申请量为: 42.69966678993407
agent 16 在 53999 次迭代中的资源申请量为: 48.92613135139638
agent 17 在 53999 次迭代中的资源申请量为: 49.43155578240953
agent 18 在 53999 次迭代中的资源申请量为: 46.07652993086456
agent 19 在 53999 次迭代中的资源申请量为: 45.567533127789474
agent 20 在 53999 次迭代中的资源申请量为: 49.99068002375894
agent 21 在 53999 次迭代中的资源申请量为: 6

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 55.585782035170666
agent 2 在 55999 次迭代中的资源申请量为: 55.550194628302336
agent 3 在 55999 次迭代中的资源申请量为: 60.80219082460608
agent 4 在 55999 次迭代中的资源申请量为: 39.49236966792236
agent 5 在 55999 次迭代中的资源申请量为: 56.51644327436709
agent 6 在 55999 次迭代中的资源申请量为: 54.440400454780374
agent 7 在 55999 次迭代中的资源申请量为: 66.27056354488175
agent 8 在 55999 次迭代中的资源申请量为: 46.572209168620994
agent 9 在 55999 次迭代中的资源申请量为: 65.25608483388052
agent 10 在 55999 次迭代中的资源申请量为: 54.41164702122708
agent 11 在 55999 次迭代中的资源申请量为: 66.54151327101513
agent 12 在 55999 次迭代中的资源申请量为: 38.00206975148434
agent 13 在 55999 次迭代中的资源申请量为: 50.971346255911754
agent 14 在 55999 次迭代中的资源申请量为: 41.36559979337059
agent 15 在 55999 次迭代中的资源申请量为: 42.69966678996795
agent 16 在 55999 次迭代中的资源申请量为: 48.92613135143586
agent 17 在 55999 次迭代中的资源申请量为: 49.431555782449394
agent 18 在 55999 次迭代中的资源申请量为: 46.07652993090196
agent 19 在 55999 次迭代中的资源申请量为: 45.56753312782604
agent 20 在 55999 次迭代中的资源申请量为: 49.99068002379932
agent 21 在 55999 次迭代中的资源申请量为

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 55.585782035215864
agent 2 在 57999 次迭代中的资源申请量为: 55.5501946283472
agent 3 在 57999 次迭代中的资源申请量为: 60.8021908246556
agent 4 在 57999 次迭代中的资源申请量为: 39.49236966795297
agent 5 在 57999 次迭代中的资源申请量为: 56.51644327441328
agent 6 在 57999 次迭代中的资源申请量为: 54.44040045482468
agent 7 在 57999 次迭代中的资源申请量为: 66.27056354493573
agent 8 在 57999 次迭代中的资源申请量为: 46.572209168658134
agent 9 在 57999 次迭代中的资源申请量为: 65.25608483393385
agent 10 在 57999 次迭代中的资源申请量为: 54.41164702127084
agent 11 在 57999 次迭代中的资源申请量为: 66.54151327106948
agent 12 在 57999 次迭代中的资源申请量为: 38.00206975151512
agent 13 在 57999 次迭代中的资源申请量为: 50.97134625595263
agent 14 在 57999 次迭代中的资源申请量为: 41.36559979340345
agent 15 在 57999 次迭代中的资源申请量为: 42.699666790001835
agent 16 在 57999 次迭代中的资源申请量为: 48.926131351475355
agent 17 在 57999 次迭代中的资源申请量为: 49.431555782489255
agent 18 在 57999 次迭代中的资源申请量为: 46.07652993093936
agent 19 在 57999 次迭代中的资源申请量为: 45.567533127862596
agent 20 在 57999 次迭代中的资源申请量为: 49.9906800238397
agent 21 在 57999 次迭代中的资源申请量为: 6

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 55.58578203526105
agent 2 在 59999 次迭代中的资源申请量为: 55.550194628392056
agent 3 在 59999 次迭代中的资源申请量为: 60.802190824705114
agent 4 在 59999 次迭代中的资源申请量为: 39.49236966798357
agent 5 在 59999 次迭代中的资源申请量为: 56.51644327445948
agent 6 在 59999 次迭代中的资源申请量为: 54.440400454868985
agent 7 在 59999 次迭代中的资源申请量为: 66.2705635449897
agent 8 在 59999 次迭代中的资源申请量为: 46.57220916869527
agent 9 在 59999 次迭代中的资源申请量为: 65.25608483398719
agent 10 在 59999 次迭代中的资源申请量为: 54.411647021314586
agent 11 在 59999 次迭代中的资源申请量为: 66.54151327112385
agent 12 在 59999 次迭代中的资源申请量为: 38.0020697515459
agent 13 在 59999 次迭代中的资源申请量为: 50.971346255993524
agent 14 在 59999 次迭代中的资源申请量为: 41.36559979343629
agent 15 在 59999 次迭代中的资源申请量为: 42.69966679003571
agent 16 在 59999 次迭代中的资源申请量为: 48.92613135151485
agent 17 在 59999 次迭代中的资源申请量为: 49.43155578252912
agent 18 在 59999 次迭代中的资源申请量为: 46.07652993097676
agent 19 在 59999 次迭代中的资源申请量为: 45.56753312789916
agent 20 在 59999 次迭代中的资源申请量为: 49.99068002388009
agent 21 在 59999 次迭代中的资源申请量为: 6